# Partial wave analysis

In [1]:
using JLD
using Plots

In [2]:
push!(LOAD_PATH,pwd()*"\\src")

3-element Array{Any,1}:
 "C:\\Users\\mikha\\AppData\\Local\\Julia-0.6.1\\local\\share\\julia\\site\\v0.6"
 "C:\\Users\\mikha\\AppData\\Local\\Julia-0.6.1\\share\\julia\\site\\v0.6"       
 "C:\\Users\\mikha\\Documents\\pwa_from_scratch\\src"                            

In [3]:
# workspace()
using DalitzPlotAnalysis
using amplitudes_compass

In [4]:
# mm = readdlm("C:\\Users\\mikha\\cernbox\\tmp\\2300.mc.txt");

In [5]:
function precalculate_compass_basis(fin,fout)
    mm = readdlm(fin);
    m2 = [COMPASS_wave(i,mm[e,:]...) for e=1:size(mm,1), i=1:88];
#     writedlm(fout*".re", real(m2))
#     writedlm(fout*".im", imag(m2))
    save(fout,"real",real(m2),"imag",imag(m2))
end
function read_precalc_basis(fname)
    ld = load(fname)
    ld["real"]+1im*ld["imag"]
end

read_precalc_basis (generic function with 1 method)

In [6]:
# @time let mm = readdlm("C:\\Users\\mikha\\cernbox\\tmp\\2300.mc.txt");
#     m2 = [COMPASS_wave(i,mm[e,:]...) for e=1:size(mm,1), i=1:88];
#     writedlm("mc.re.txt", real(m2))
#     writedlm("mc.im.txt", imag(m2))
# end

In [7]:
# @time precalculate_compass_basis("C:\\Users\\mikha\\cernbox\\tmp\\2300.mc.txt", "mc.jld")
# @time precalculate_compass_basis("C:\\Users\\mikha\\cernbox\\tmp\\2300.rd.txt", "rd.jld")

In [8]:
const MC = read_precalc_basis("mc.jld");

In [9]:
@time sum_mat = [sum(MC[e,i]'*MC[e,j] for e in 1:size(MC,1)) for i=1:88, j=1:88]/size(MC,1);

  7.375886 seconds (117.64 k allocations: 5.656 MiB)


In [10]:
sum_mat_n = [sum_mat[i,j]/sqrt(sum_mat[i,i]*sum_mat[j,j]) for i=1:88, j=1:88];

In [11]:
# heatmap(real(sum_mat_n))

In [12]:
const Bmat = sum_mat;
const BF = read_precalc_basis("rd.jld") # [1:10000,:]
const Nd = size(BF,1);

### Coherence matrix

In [13]:
waves = readdlm(pwd()*"/src/wavelist_formated.txt");

In [14]:
noϵ = [i==1 for i=1:size(waves[:,6],1)]
posϵ = [ϵ=="+" for ϵ in waves[:,6]]
negϵ = [ϵ=="-" for ϵ in waves[:,6]]
## decomp
sum(noϵ)+sum(posϵ)+sum(negϵ)==length(waves[:,6])
block(i) = 1*noϵ[i]+2*posϵ[i]+3*negϵ[i]

block (generic function with 1 method)

In [15]:
COH = [block(i)==block(j) for i=1:88, j=1:88];
const COHc = convert(Array{Complex{Float64},2},COH);

### Map of parameters

In [16]:
Trel = let vect=[], blocks=[noϵ,posϵ,negϵ,negϵ]
    temp = []
    numb = []
    for bl in blocks
        push!(temp,false)
        push!(temp,[true for i=1:(sum(bl)-1)]...)
        push!(numb,collect(1:88)[bl]...)
    end 
    Trel = zeros(Complex{Float64},88,sum(temp+1))
    count=1
    for (i,b) in enumerate(temp)
        Trel[numb[i],count] = 1.0;
        count+=1
        if b
            Trel[numb[i],count] = 1.0im; count+=1
        end
    end
    Trel
#     numb
end;

In [17]:
const TT = Trel;

In [18]:
function get_form(B)
    @inbounds m = B.*TT;
    @inbounds v = m'COHc*m
    v
end
# @time const BFE = get_form.([BF[e,:] for e=1:size(BF,1)]);

In [19]:
@time const TBmatT = TT'*(Bmat.*COHc)*TT;

  1.755670 seconds (1.34 M allocations: 65.993 MiB, 5.15% gc time)


In [20]:
sR = open("recalc1.re.bin","w+") # or "r"
sI = open("recalc1.im.bin","w+") # or "r"
Ar = Mmap.mmap(sR, Matrix{Float64}, (size(TT,2)*Nd, size(TT,2)));
Am = Mmap.mmap(sI, Matrix{Float64}, (size(TT,2)*Nd, size(TT,2)));

In [21]:
@time let d = size(TT,2)
    for e = 1:Nd
        matr = get_form(BF[e,:])
        Ar[((e-1)*d+1):(e*d),:] .= real(matr)
        Am[((e-1)*d+1):(e*d),:] .= imag(matr)        
    end
end

1056.400267 seconds (7.21 M allocations: 125.144 GiB, 0.80% gc time)


In [22]:
function preBFE(e)
    d = size(TT,2)
    Ar[((e-1)*d+1):(e*d),:] .+ 1im*Am[((e-1)*d+1):(e*d),:]
end

preBFE (generic function with 1 method)

In [38]:
@time for i in 1:10000
    preBFE(i);
end

 71.228806 seconds (500.00 k allocations: 15.486 GiB, 0.94% gc time)


Likelihood functions

In [31]:
function LLH(pars::Vector{Float64})
    res = sum(log(real(begin
                    m = get_form(BF[e,:])
                    pars'*m*pars
                    end)) for e in 1:Nd)
    - res + real(pars'*TBmatT*pars) * Nd
end
function dLLH(pars::Vector{Float64})
    resP = sum(2real(begin
                    m = get_form(BF[e,:])
                    v = m*pars
                    v / (pars'*v)
                end)  for e in 1:Nd)
    - resP + 2real(TBmatT*pars)* Nd 
end
function LLH_and_GRAD!(pars::Vector{Float64}, grad::Vector{Float64})
    val = 0.0; grad .= 0.0;
    for e in 1:Nd
        m = get_form(BF[e,:])
        v = m*pars
        sc = real(pars'*v)
        val -= log(sc)
        grad .-= (2real(v) ./ sc)
    end
#     println(val)
    BB = TBmatT*pars;
    val += real(pars'*BB) * Nd;
    grad .+= 2real(BB)* Nd;
    return val;
end

LLH_and_GRAD! (generic function with 1 method)

In [23]:
function LLH(pars::Vector{Float64})
    res = sum(log(real(pars'*preBFE(e)*pars)) for e in 1:Nd)
    - res + real(pars'*TBmatT*pars) * Nd
end
function dLLH(pars::Vector{Float64})
    resP = sum(2real(begin
                    v = preBFE(e)*pars
                    v / (pars'*v)
                end)  for e in 1:Nd)
    - resP + 2real(TBmatT*pars)* Nd 
end
function LLH_and_GRAD!(pars::Vector{Float64}, grad::Vector{Float64})
    val = 0.0; grad .= 0.0;
    for e in 1:Nd
        v = preBFE(e)*pars
        sc = real(pars'*v)
        val -= log(sc)
        grad .-= (2real(v) ./ sc)
    end
#     println(val)
    BB = TBmatT*pars;
    val += real(pars'*BB) * Nd;
    grad .+= 2real(BB)* Nd;
    return val;
end

LLH_and_GRAD! (generic function with 1 method)

In [24]:
test_t = rand(size(TT,2));

In [32]:
@time let val = 0.0, g=fill(0.0, size(TT,2))
    LLH_and_GRAD!(test_t,g);
    g
end

167.301354 seconds (1.51 M allocations: 83.345 GiB, 2.05% gc time)


186-element Array{Float64,1}:
  50719.5      
      1.50939e6
      2.78119e5
      2.89703e5
      5.56113e5
      4.08507e5
      2.29301e6
      2.5627e6 
 -28475.2      
      1.10907e5
  33889.3      
  33740.8      
      2.39716e6
      ⋮        
      2.14281e6
      8.60932e5
      4.35032e6
      1.38378e6
      2.16271e6
      2.81342e6
  17701.4      
  -6718.17     
  21686.7      
  59574.2      
  41047.1      
  57044.5      

In [30]:
@time LLH(test_t)

166.413104 seconds (1.33 M allocations: 83.095 GiB, 1.91% gc time)


2.5204232625420623e7

In [27]:
@time dLLH(test_t)

669.262460 seconds (5.19 M allocations: 128.404 GiB, 0.94% gc time)


186-element Array{Float64,1}:
  50719.5      
      1.50939e6
      2.78119e5
      2.89703e5
      5.56113e5
      4.08507e5
      2.29301e6
      2.5627e6 
 -28475.2      
      1.10907e5
  33889.3      
  33740.8      
      2.39716e6
      ⋮        
      2.14281e6
      8.60932e5
      4.35032e6
      1.38378e6
      2.16271e6
      2.81342e6
  17701.4      
  -6718.17     
  21686.7      
  59574.2      
  41047.1      
  57044.5      

In [39]:
using NLopt

In [41]:
minpars0 = vcat(readdlm("min.txt")...);

In [42]:
function minimize(; verbose=0)
#     function abs_inverse(x::Vector, grad::Vector) 
#         v = LLH(x)
#         if length(grad) > 0
#             grad[:] = dLLH(x)
#         end        
#         verbose==1 && println(v)
#         verbose==2 && println("\n------\n$x\n$grad\n->$v")
#         v
#     end
    function abs_inverse(x::Vector, grad::Vector) 
        if length(grad) > 0
            v = LLH_and_GRAD!(x,grad)
        else
            v = LLH(x)
        end
        verbose==1 && println(v)
        verbose==2 && println("\n------\n$x\n$grad\n->$v")
        return v;
    end

    # find minimum which of course suppose t o be zero
#     opt = Opt(:LN_COBYLA, 88)
    opt = Opt(:LD_LBFGS, size(TT,2)) # try LD_LBFGS || LD_MMA
    xtol_rel!(opt,1e-4)
#     set_maxeval(opt,500000)

    min_objective!(opt, abs_inverse)

    (minf,pars,ret) = optimize(opt, minpars0)#rand(size(TT,2)))
    println("got $minf at $pars = [m, Γ] after some iterations (returned $ret)")
    pars
end

minimize (generic function with 1 method)

In [122]:
# hcat(minpar0,minpars)

In [ ]:
@time minpars = minimize(verbose=1)

23366.296420494677
1.9326007335985876e16
1.9326498060614155e12
1.9328739063007024e8
25998.38820035587
20599.473170309677
19637.54012927848
18699.457426628927
18336.088724124733
17993.53692511574
17432.557759213
16987.726369652883
16617.622887279635
16102.84898749672
15743.439161766582
15502.853275623565
15283.82461531779
14968.343243420502
14870.700887613639
14668.041998644461
14520.388205137904
14310.20063805615
14065.626130301272
13877.213806334534
13752.00720966702
13632.389455453958
13529.848447862081
13415.662301787263
13274.816985189245
13146.663674966461
13041.892875899037
12943.744038848468
12857.958010030474
12776.4415372746
12685.232445403657
12596.850226538547
12519.886100685704
12434.693302502885
12367.42901549369
12323.854187817764
12295.191823465604
12263.073669191494
12218.223409129292
12165.546292278072
12116.267823363814
12081.594792919335
12046.481256047598
12017.615135404456
11990.746924433886
11959.19113226021
11924.691528510753
11895.123381786863
11860.701626866008

In [125]:
@time minpars2 = minimize(verbose=1)

1985.6664735858049
2.545538699031223e13
2.5463758591544294e9
244231.61305025243
1735.9946015188343
1815.056597819199
1611.2839737045306
1533.4917685024902
1443.0401155377403
1387.5537194460467
1342.4531660343437
1306.373855693575
1274.94897115582
1348.0614679457867
1235.037570893297
1201.0837915566517
1172.1819448585848
1148.6675308011472
1123.5731329235114
1093.895893837178
1071.005482047538
1051.6170285032313
1031.9907272618384
1011.937012399032
991.6654161120823
977.0672867494777
965.3305501621289
957.9418935001995
946.1790157981886
936.8134814615441
927.0641843046324
913.0655257620438
899.2983625612615
884.0393958211534
872.8886446277738
864.425013694161
860.473356879771
852.9020156152783
846.0303376622614
838.0854593360527
829.4465442206983
821.1319878807972
815.5100258219791
808.7673494110986
802.1643914148717
796.902895553947
791.2391120320481
784.035398039694
772.6475671366024
764.7038600845644
756.035074968926
749.4456683983608
745.2630464903959
739.1387479707319
729.021757077

487.56412053817803
487.56397394222586
487.5638446271387
487.56372634456056
487.5635755206131
487.5634324256316
487.56327208454786
487.5631192453966
487.5629780804502
487.5627984495095
487.5625436740738
487.56215008737126
487.56162145161943
487.56108977326767
487.56061367201073
487.56007056616545
487.5593673246385
487.55851586764584
487.55760824169374
487.55663987956177
487.555550042769
487.5542695136919
487.55328719432146
487.5524862831353
487.55145835123403
487.55001022657234
487.54780549311
487.5455184816037
487.54352121464035
487.54148834071566
487.53882349848936
487.5356775018772
487.53353003398115
487.53233968487075
487.53140513526523
487.53017032444586
487.52860622039043
487.527233922121
487.52601624853196
487.5249185461016
487.52389389611017
487.5231262977668
487.5225914865787
487.522151675008
487.52167842281233
487.5214010740656
487.5212989757856
487.5212328581347
487.5211516044892
487.5210523651276
487.5209712888809
487.52091009346805
487.5208584568354
487.5207993377335
487.52

186-element Array{Float64,1}:
 -5.96031e-35
 -0.0332271  
 -0.0491032  
  0.0352418  
 -0.0722667  
  0.000656321
  0.0310938  
 -0.0428372  
  0.121539   
 -0.0329123  
 -0.11297    
 -0.137733   
 -0.0210682  
  ⋮          
  0.136467   
 -0.143576   
 -0.00213442 
 -0.464127   
 -0.0847298  
  0.240082   
 -0.282778   
  0.207908   
 -0.101294   
 -0.130073   
 -0.139424   
 -0.253787   

In [126]:
const MCD = readdlm("C:\\Users\\mikha\\cernbox\\tmp\\2300.mc.txt");

In [131]:
@time weights = let mpars = TT*minpars2
    [real(begin
                v = MC[i,:].*mpars
                v'*COH*v
                end) for i in 1:size(MC,1)];
    end;

 10.771764 seconds (5.18 M allocations: 2.468 GiB, 1.61% gc time)


In [132]:
histogram(sqrt.(MCD[:,2]), weights=weights, bins=(linspace(0.3,2.2,100)))

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0.5 
 
 
 1.0 
 
 
 1.5 
 
 
 2.0 
 
 
 0 
 
 
 2500 
 
 
 5000 
 
 
 7500 
 
 
 10000 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 y1

In [133]:
DT = readdlm("C:\\Users\\mikha\\cernbox\\tmp\\2300.rd.txt");

In [134]:
stephist(sqrt.(MCD[:,2]), weights=weights/size(MCD,1)*size(DT,1),
    bins=linspace(0.3,2.2,100),lab="weighted MC")
stephist!(sqrt.(DT[:,2]), bins=linspace(0.3,2.2,100),lab="data") 

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0.5 
 
 
 1.0 
 
 
 1.5 
 
 
 2.0 
 
 
 0 
 
 
 500 
 
 
 1000 
 
 
 1500 
 
 
 2000 
 
<polyline clip-path="url(#clip0602)" style="stroke:#009af9; stroke-width:1; stroke-opacity:1; fill:none" points="
 42.5104,375.869 42.5104,359.932 47.9427,359.932 47.9427,353.486 53.3751,353.486 53.3751,352.48 58.8075,352.48 58.8075,345.479 64.2398,345.479 64.2398,344.835 
 69.6722,344.835 69.6722,341.438 75.1046,341.438 75.1046,332.023 80.537,332.023 80.537,326.825 85.9693,326.825 85.9693,325.351 91.4017,325.351 91.4017,322.002 
 96.8341,322.002 96.8341,314.397 102.266,314.397 102.266,308.161 107.699,308.161 107.699,303.595 113.131,303.595 113.131,299.025 118.564,299.025 118.564,295.187 
 123.996,295.187 123.996,287.651 129.428,287.651 129.428,271.06 134.861,271.06 134.861,273.856 140.293,273.856 140.293,250.391 145.725,250.391 145.725,227.108 
 151.158,227.108 151.158,214.362 156.59,214.362 156.59,188.71 162.023,188.71 162.023,140.52 167.455,140.52 167.455,93.4565 172.887,93.4565 172.887,62.9839 
 178.32,62.9839 178.32,69.3383 183.752,69.3383 183.752,104.633 189.184,104.633 189.184,129.784 194.617,129.784 194.617,175.086 200.049,175.086 200.049,201.368 
 205.481,201.368 205.481,206.224 210.914,206.224 210.914,212.95 216.346,212.95 216.346,239.396 221.779,239.396 221.779,232.72 227.211,232.72 227.211,229.203 
 232.643,229.203 232.643,232.921 238.076,232.921 238.076,257.792 243.508,257.792 243.508,243.62 248.94,243.62 248.94,233.202 254.373,233.202 254.373,238.043 
 259.805,238.043 259.805,233.316 265.238,233.316 265.238,224.086 270.67,224.086 270.67,230.414 276.102,230.414 276.102,224.283 281.535,224.283 281.535,209.461 
 286.967,209.461 286.967,204.394 292.399,204.394 292.399,190.715 297.832,190.715 297.832,180.374 303.264,180.374 303.264,171.13 308.696,171.13 308.696,162.447 
 314.129,162.447 314.129,159.22 319.561,159.22 319.561,165.464 324.994,165.464 324.994,176.758 330.426,176.758 330.426,189.431 335.858,189.431 335.858,209.308 
 341.291,209.308 341.291,226.3 346.723,226.3 346.723,235.263 352.155,235.263 352.155,243.607 357.588,243.607 357.588,244.485 363.02,244.485 363.02,249.688 
 368.453,249.688 368.453,249.46 373.885,249.46 373.885,252.333 379.317,252.333 379.317,256.775 384.75,256.775 384.75,254.863 390.182,254.863 390.182,254.863 
 395.614,254.863 395.614,249.305 401.047,249.305 401.047,255.208 406.479,255.208 406.479,245.235 411.912,245.235 411.912,237.398 417.344,237.398 417.344,232.835 
 422.776,232.835 422.776,227.586 428.209,227.586 428.209,225.53 433.641,225.53 433.641,230.185 439.073,230.185 439.073,229.536 444.506,229.536 444.506,216.929 
 449.938,216.929 449.938,221.138 455.37,221.138 455.37,218.915 460.803,218.915 460.803,225.741 466.235,225.741 466.235,214.987 471.668,214.987 471.668,213.121 
 477.1,213.121 477.1,219.976 482.532,219.976 482.532,220.51 487.965,220.51 487.965,224.158 493.397,224.158 493.397,229.867 498.829,229.867 498.829,238.27 
 504.262,238.27 504.262,236.91 509.694,236.91 509.694,237.693 515.127,237.693 515.127,249.067 520.559,249.067 520.559,257.31 525.991,257.31 525.991,245.752 
 531.424,245.752 531.424,248.89 536.856,248.89 536.856,247.412 542.288,247.412 542.288,253.993 547.721,253.993 547.721,270.767 553.153,270.767 553.153,264.839 
 558.585,264.839 558.585,311.321 564.018,311.321 564.018,344.695 569.45,344.695 569.45,373.81 574.883,373.81 574.883,375.869 580.315,375.869 
 "/>
<polyline clip-path="url(#clip0602)" style="stroke:#e26f46; stroke-width:1; stroke-opacity:1; fill:none" points="
 42.5104,375.869 42.5104,357.625 47.9427,357.625 47.9427,351.11 53.3751,351.11 53.3751,347.037 58.8075,347.037 58.8075,339.381 64.2398,339.381 64.2398,334.821 
 69.6722,334.821 69.6722,331.563 75.1046,331.563 75.1046,327.816 80.537,327.816 80.537,318.857 85.9693,318.857 85.9693,311.527 91.4017,311.527 91.4017,301.265 
 96.8341,301.265 96.8341,305.012 102.266,305.012 102.266,297.845 107.699,297.

In [135]:
stephist(MCD[:,3], weights=weights/size(MCD,1)*size(DT,1),
    bins=linspace(-1,1,100),lab="weighted MC")
stephist!(DT[:,3], bins=linspace(-1,1,100),lab="data") 

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 -1.0 
 
 
 -0.5 
 
 
 0.0 
 
 
 0.5 
 
 
 1.0 
 
 
 0 
 
 
 1000 
 
 
 2000 
 
 
 3000 
 
 
 4000 
 
 
 5000 
 
 
 6000 
 
<polyline clip-path="url(#clip0802)" style="stroke:#009af9; stroke-width:1; stroke-opacity:1; fill:none" points="
 42.5104,375.869 42.5104,156.457 47.9427,156.457 47.9427,193.086 53.3751,193.086 53.3751,227.089 58.8075,227.089 58.8075,261.651 64.2398,261.651 64.2398,280.345 
 69.6722,280.345 69.6722,300.25 75.1046,300.25 75.1046,312.104 80.537,312.104 80.537,319.061 85.9693,319.061 85.9693,327.154 91.4017,327.154 91.4017,332.818 
 96.8341,332.818 96.8341,336.267 102.266,336.267 102.266,338.973 107.699,338.973 107.699,342.43 113.131,342.43 113.131,342.166 118.564,342.166 118.564,343.427 
 123.996,343.427 123.996,344.3 129.428,344.3 129.428,343.198 134.861,343.198 134.861,343.191 140.293,343.191 140.293,344.721 145.725,344.721 145.725,342.383 
 151.158,342.383 151.158,344.293 156.59,344.293 156.59,343.175 162.023,343.175 162.023,343.509 167.455,343.509 167.455,342.091 172.887,342.091 172.887,343.307 
 178.32,343.307 178.32,344.239 183.752,344.239 183.752,344.591 189.184,344.591 189.184,345.753 194.617,345.753 194.617,347.058 200.049,347.058 200.049,343.421 
 205.481,343.421 205.481,345.374 210.914,345.374 210.914,346.433 216.346,346.433 216.346,344.743 221.779,344.743 221.779,345.624 227.211,345.624 227.211,344.821 
 232.643,344.821 232.643,347.76 238.076,347.76 238.076,345.904 243.508,345.904 243.508,346.52 248.94,346.52 248.94,345.404 254.373,345.404 254.373,345.721 
 259.805,345.721 259.805,345.916 265.238,345.916 265.238,345.706 270.67,345.706 270.67,344.671 276.102,344.671 276.102,347.026 281.535,347.026 281.535,348.204 
 286.967,348.204 286.967,345.242 292.399,345.242 292.399,342.879 297.832,342.879 297.832,347.152 303.264,347.152 303.264,348.032 308.696,348.032 308.696,344.029 
 314.129,344.029 314.129,347.828 319.561,347.828 319.561,348.024 324.994,348.024 324.994,346.55 330.426,346.55 330.426,347.769 335.858,347.769 335.858,347.972 
 341.291,347.972 341.291,351.279 346.723,351.279 346.723,346.015 352.155,346.015 352.155,348.324 357.588,348.324 357.588,350.408 363.02,350.408 363.02,349.234 
 368.453,349.234 368.453,347.06 373.885,347.06 373.885,345.464 379.317,345.464 379.317,345.416 384.75,345.416 384.75,343.166 390.182,343.166 390.182,346.568 
 395.614,346.568 395.614,349.308 401.047,349.308 401.047,343.768 406.479,343.768 406.479,344.843 411.912,344.843 411.912,345.2 417.344,345.2 417.344,344.053 
 422.776,344.053 422.776,342.353 428.209,342.353 428.209,339.947 433.641,339.947 433.641,340.843 439.073,340.843 439.073,339.448 444.506,339.448 444.506,338.042 
 449.938,338.042 449.938,338.548 455.37,338.548 455.37,336.378 460.803,336.378 460.803,333.557 466.235,333.557 466.235,335.461 471.668,335.461 471.668,336.24 
 477.1,336.24 477.1,334.434 482.532,334.434 482.532,334 487.965,334 487.965,332.505 493.397,332.505 493.397,333.49 498.829,333.49 498.829,331.834 
 504.262,331.834 504.262,325.673 509.694,325.673 509.694,324.824 515.127,324.824 515.127,320.842 520.559,320.842 520.559,317.119 525.991,317.119 525.991,307.854 
 531.424,307.854 531.424,306.328 536.856,306.328 536.856,300.043 542.288,300.043 542.288,292.862 547.721,292.862 547.721,272.011 553.153,272.011 553.153,268.41 
 558.585,268.41 558.585,243.774 564.018,243.774 564.018,225.809 569.45,225.809 569.45,197.935 574.883,197.935 574.883,128.6 580.315,128.6 580.315,375.869 
 
 "/>
<polyline clip-path="url(#clip0802)" style="stroke:#e26f46; stroke-width:1; stroke-opacity:1; fill:none" points="
 42.5104,375.869 42.5104,110.432 47.9427,110.432 47.9427,134.357 53.3751,134.357 53.3751,190.021 58.8075,190.021 58.8075,224.924 64.2398,224.924 64.2398,256.962 
 69.6722,256.962 69.6722,282.14 75.1046,282.14 75.1046,295.385 80.537,295.385 80.537,312.687 85.9693,312.687 85.9693,317.34 91.4017,317.34 91.4017,323.068 
 96.8341,323.0